<a href="https://colab.research.google.com/github/naufalkr/aic/blob/main/Flask-server/nlp%20model/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PySastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 5.1 MB/s eta 0:00:00


In [42]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import os
import pickle
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

#### Import Dataset

In [45]:
# data_path = '/content/dataset'
data_path = '...\Flask-server\dataset'

labels = []
tokens = []
titles = []

for f in os.listdir(data_path):
    full_path = os.path.join(data_path, f)
    if os.path.isfile(full_path):
        titles.append(f.split('.')[0])
        with open(full_path, 'r', encoding='utf8', errors='ignore') as infile:
            for line in infile:
                tokens.append(line.strip())
                labels.append(len(titles) - 1)

df = pd.DataFrame(list(zip(tokens, labels)), columns=['sent', 'label'])

In [46]:
df

,sent,label
0,DevOps mengintegrasikan pengembangan perangkat...,0
1,Praktik DevOps berfokus pada otomatisasi prose...,0
2,"Dengan DevOps, tim dapat mempercepat siklus ri...",0
3,DevOps menciptakan lingkungan kerja yang lebih...,0
4,Penggunaan alat-alat seperti Jenkins dan Docke...,0
...,...,...
1002,"Dalam pengembangan aplikasi iOS, Anda harus me...",4
1003,Keterampilan dalam pemrograman iOS memberi And...,4
1004,Pemrograman iOS melibatkan pembuatan aplikasi ...,4
1005,"Dengan keterampilan dalam pemrograman iOS, And...",4


In [47]:
titles

['devops', 'backend', 'frontend', 'android', 'ios']

In [48]:
df.sample(20)

,sent,label
899,"Dalam pemrograman iOS, Anda harus memahami car...",4
657,Menggunakan Jetpack Compose memungkinkan penge...,3
805,Pemrograman iOS juga mencakup pengelolaan memo...,4
879,Pemrograman iOS sering melibatkan penggunaan g...,4
845,Keterampilan dalam pemrograman iOS juga memban...,4
717,Android menyediakan framework untuk pengembang...,3
128,Penggunaan prinsip DevOps dapat meningkatkan k...,0
601,Desain antarmuka yang konsisten membantu dalam...,2
251,Pengembang backend harus memahami cara menyusu...,1
676,Pemrograman Android juga melibatkan penggunaan...,3


#### Data Cleaning

In [49]:
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

stop_factory = StopWordRemoverFactory()
stop_words = stop_factory.get_stop_words()

In [50]:
def preprocess_text(text):
    cleaned_text = " ".join([stemmer.stem(word) for word in re.sub("[^a-zA-Z]", " ", text).split() if word.lower() not in stop_words])
    return cleaned_text.lower()

df['cleaned'] = df['sent'].apply(preprocess_text)


In [51]:
df

,sent,label,cleaned
0,DevOps mengintegrasikan pengembangan perangkat...,0,devops integrasi kembang perangkat lunak opera...
1,Praktik DevOps berfokus pada otomatisasi prose...,0,praktik devops fokus otomatisasi proses kemban...
2,"Dengan DevOps, tim dapat mempercepat siklus ri...",0,devops tim cepat siklus rilis perangkat lunak ...
3,DevOps menciptakan lingkungan kerja yang lebih...,0,devops cipta lingkung kolaboratif kembang tim ...
4,Penggunaan alat-alat seperti Jenkins dan Docke...,0,guna alat alat jenkins docker implementasi pri...
...,...,...,...
1002,"Dalam pengembangan aplikasi iOS, Anda harus me...",4,kembang aplikasi ios paham alat framework bang...
1003,Keterampilan dalam pemrograman iOS memberi And...,4,terampil pemrograman ios mampu teknologi baru ...
1004,Pemrograman iOS melibatkan pembuatan aplikasi ...,4,pemrograman ios libat buat aplikasi manfaat fi...
1005,"Dengan keterampilan dalam pemrograman iOS, And...",4,terampil pemrograman ios kembang aplikasi manf...


In [68]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

tfidf = TfidfVectorizer(max_features=1000)

X_tfidf = tfidf.fit_transform(df['cleaned'])

selector = SelectKBest(chi2, k=800)
X_selected = selector.fit_transform(X_tfidf, df['label'])

In [69]:
print(X_tfidf.shape)

(1007, 872)


In [70]:
print(X_selected.shape)

(1007, 800)


In [71]:
x_train, x_test, y_train, y_test = train_test_split(X_selected, df['label'], test_size=0.2, random_state=42, stratify=df['label'])

In [72]:
model = LogisticRegression(max_iter=1000)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('select', SelectKBest(chi2, k=200)),
    ('classifier', LogisticRegression(max_iter=1000))
])

In [73]:
pipeline.fit(df['cleaned'], df['label'])
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [74]:
y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        45
           1       1.00      1.00      1.00        43
           2       1.00      1.00      1.00        39
           3       1.00      1.00      1.00        33
           4       1.00      1.00      1.00        42

    accuracy                           1.00       202
   macro avg       1.00      1.00      1.00       202
weighted avg       1.00      1.00      1.00       202

[[45  0  0  0  0]
 [ 0 43  0  0  0]
 [ 0  0 39  0  0]
 [ 0  0  0 33  0]
 [ 0  0  0  0 42]]


In [75]:
def predict_text(text):
    cleaned_text = preprocess_text(text)
    prediction = pipeline.predict([cleaned_text])
    return titles[prediction[0]]

In [76]:
new_text = "saya mempunyai pengalaman melakukan pemrograman pada bahasa swift"
predicted_label = predict_text(new_text)
print(predicted_label)

ios


In [77]:
new_text = "bapak jokowi adalah idola saya. saya ingin bisa menjadi pemrogram yang bekerja di bidang server seperti bapak jokowi"
predicted_label = predict_text(new_text)
print(predicted_label)

backend
